In [1]:
# ─── Cell 1: Configuration ────────────────────────────────────────────
COURSE_FILE       = "data_course_for_RSv2.xlsx"
JOBS_FILE         = "combined_all_jobs_requirements.xlsx"
SHEET_NAME        = "Sheet1"

# ─── Learner Profile ─────────────────────────────────────────────────
# DESIRED_JOB       = "นิติกร"
COURSES_COMPLETED = [
    # add more course names here exactly as in your NAME_T column
]

# ─── Embedding Params & Stopwords ────────────────────────────────────
CHUNK_SIZE        = 64
thai_stopwords    = {
}


In [2]:
# ─── Imports & Helper Functions ─────────────────────────────────
import pandas as pd
import re
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    tokens = word_tokenize(str(text))
    return [w for w in tokens if w not in thai_stopwords]

def normalize(text: str) -> str:
    text = text.replace("\xa0", " ")
    return re.sub(r"\s+", " ", text).strip()


c:\Users\ninen\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ─── Load & Preprocess Data ───────────────────────────────────────
courses = pd.read_excel(COURSE_FILE, sheet_name=SHEET_NAME)
jobs    = pd.read_excel(JOBS_FILE)
jobs    = jobs.rename(columns={"Job Title":"JOB_NAME", "Job Description":"REQUIREMENTS"})

courses["PROCESSED_DESC"] = courses["C_Description"].apply(preprocess_text)
jobs   ["PROCESSED_REQ"]  = jobs  ["REQUIREMENTS"].apply(preprocess_text)


In [4]:
# # ─── Load Model & Tokenizer ───────────────────────────────────────
# device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
# model     = AutoModel.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")\
#                 .to(device).eval()
# vocab_size = model.config.vocab_size


In [5]:
# ─── Cell 5: TF-IDF + SVD embeddings ────────────────────────────────────
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition      import TruncatedSVD
from scipy import sparse

# 1) Build TEXT_CLEAN columns if you haven’t already
jobs   ['TEXT_CLEAN'] = jobs   ['PROCESSED_REQ'].apply(lambda toks: ' '.join(toks))
courses['TEXT_CLEAN'] = courses['PROCESSED_DESC'].apply(lambda toks: ' '.join(toks))

# 2) Fit TF-IDF on all texts
vectorizer = TfidfVectorizer(max_features=5000)
all_texts   = pd.concat([jobs['TEXT_CLEAN'], courses['TEXT_CLEAN']])
vectorizer.fit(all_texts)

# 3) Transform to sparse matrices
jobs_tfidf    = vectorizer.transform(jobs['TEXT_CLEAN'])
courses_tfidf = vectorizer.transform(courses['TEXT_CLEAN'])

# 4) (Optional) Dimensionality reduction
svd = TruncatedSVD(n_components=300, random_state=42)
# stack the two matrices so that SVD learns joint components
combined = sparse.vstack([jobs_tfidf, courses_tfidf])
svd.fit(combined)

jobs_embs    = svd.transform(jobs_tfidf)
courses_embs = svd.transform(courses_tfidf)

# 5) Attach back to your DataFrames
jobs   ['EMBEDDING'] = list(jobs_embs)
courses['EMBEDDING'] = list(courses_embs)


In [10]:
learner_profile = {
    "desired_job": "นักโภชนาการ",
    "courses_completed": [
        "จริยธรรมเพื่อการดำเนินชีวิต",
        "สังคมไทยและโลกาภิวัตน์",
        "พฤติกรรมสุขภาพ",
        "การอ่านและการเขียนภาษาอังกฤษทั่วไป",
        "การบริหารงานสาธารณสุข",
        "ชีววิทยาเบื้องต้น",
        "การพัฒนาทักษะการคิด",
        "การสร้างเสริมสุขภาพ",
        "การวิจัยทางสาธารณสุข",
        "สังคมวิทยาและมานุษยวิทยาสาธารณสุข",
    ]
}

In [11]:
# ─── Cell 6b: Filter out completed courses & recommend ──────────────────
desired_job_vector = jobs.loc[jobs["JOB_NAME"] == learner_profile['desired_job'], "EMBEDDING"].iloc[0]

# 1) Exclude courses the learner has already taken
candidates = courses[~courses["C_Name"].isin(learner_profile['courses_completed'])].copy()

from sklearn.metrics.pairwise import cosine_similarity
candidates['SIMILARITY'] = candidates["EMBEDDING"].apply(
    lambda x: cosine_similarity([desired_job_vector], [x])[0][0])

# Top course recommendations
candidates_unique = candidates.drop_duplicates(subset="C_Name", keep="first")
candidates_sorted = candidates_unique.sort_values(['SIMILARITY'], ascending=False).head(10)
# 2
# 3
# 4
# 8
# 15
# 51
# 60
# 65
# 67
# 94


# Output recommendations
print("Your desired job is:")
print(learner_profile['desired_job'])
print("\nRecommended Courses:")
print(candidates_sorted[["C_Name", "C_Description", "SIMILARITY"]])


Your desired job is:
นักโภชนาการ

Recommended Courses:
                                             C_Name  \
28        วิทยาศาสตร์เพื่อคุณภาพชีวิตและสิ่งแวดล้อม   
94                                       สหกิจศึกษา   
37            กฎหมายและจริยธรรมในวิชาชีพคอมพิวเตอร์   
117                         สัมมนากระบวนการยุติธรรม   
54                              โภชนศาสตร์สาธารณสุข   
70                              การเตรียมสหกิจศึกษา   
141  กฎหมายเกี่ยวกับการกระทำความผิดของเด็กและเยาวชน   
115                            หลักวิชาชีพนักกฎหมาย   
72                          ปรัชญาและศาสนาเบื้องต้น   
142                   การฝึกประสบการณ์วิชาชีพกฎหมาย   

                                         C_Description  SIMILARITY  
28   กระบวนการและการพัฒนาวิทยาศาสตร์และเทคโนโลยีและ...    0.069876  
94   รายวิชาที่ต้องเรียนมาก่อน\n14074802  การเตรียม...    0.054310  
37    ประวัติความเป็นมาของแนวคิดเกี่ยวกับจริยธรรม น...    0.053171  
117      กระบวนการยุติธรรมที่เป็นประเด็นในท้องถิ่นแ...    0.049